In [2]:
#랜덤포레스트
#중고차 판매 이력을 다룬 데이터
#종속변수는 판매가격, 독립변후는 생산년도, 주행거리, 변속기, 마일리지 배기량 등이 있다.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url='https://media.githubusercontent.com/media/musthave-ML10/data_source/main/car.csv'
data=pd.read_csv(file_url)

In [4]:
data.head()


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [5]:
data.info()
#   name           이름
#   year           생산년 
#   selling_price  판매가
#   km_driven      주행거리
#   fuel           연료 
#   seller_type    판매자 유형
#   transmission   변속기
#   owner          차주변경내역
#   mileage        마일리지
#   engine         배기량
#   max_power      최대출력내역
#   torque         토크
#  seats           인승


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [6]:
round(data.describe(),2)

,year,selling_price,km_driven,seats
count,8128.00,8128.00,8128.00,7907.00
mean,2013.80,638271.81,69819.51,5.42
std,4.04,806253.40,56550.55,0.96
min,1983.00,29999.00,1.00,2.00
25%,2011.00,254999.00,35000.00,5.00
50%,2015.00,450000.00,60000.00,5.00
75%,2017.00,675000.00,98000.00,5.00
max,2020.00,10000000.00,2360457.00,14.00


In [ ]:
#selling_price 와 km_driven 은 아웃라이어로 보이지만 랜덤 포레스트에서는 아웃라이어를 별도로 처리하지 않습니다.

In [10]:
data['engine'].str.split()

0       [1248, CC]
1       [1498, CC]
2       [1497, CC]
3       [1396, CC]
4       [1298, CC]
           ...    
8123    [1197, CC]
8124    [1493, CC]
8125    [1248, CC]
8126    [1396, CC]
8127    [1396, CC]
Name: engine, Length: 8128, dtype: object

In [11]:
data['engine'].str.split(expand=True)

,0,1
0,1248,CC
1,1498,CC
2,1497,CC
3,1396,CC
4,1298,CC
...,...,...
8123,1197,CC
8124,1493,CC
8125,1248,CC
8126,1396,CC


In [12]:
data[['engine','engine_unit']]=data['engine'].str.split(expand=True)

In [13]:
data['engine'].head()

0    1248
1    1498
2    1497
3    1396
4    1298
Name: engine, dtype: object

In [14]:
data['engine']=data['engine'].astype('float32')

In [15]:
data['engine'].head()

0    1248.0
1    1498.0
2    1497.0
3    1396.0
4    1298.0
Name: engine, dtype: float32

In [16]:
data['engine_unit'].value_counts()

CC    7907
Name: engine_unit, dtype: int64

In [17]:
data.drop('engine_unit',axis=1,inplace=True)

In [18]:
data['max_power'].str.split()

0           [74, bhp]
1       [103.52, bhp]
2           [78, bhp]
3           [90, bhp]
4         [88.2, bhp]
            ...      
8123     [82.85, bhp]
8124       [110, bhp]
8125      [73.9, bhp]
8126        [70, bhp]
8127        [70, bhp]
Name: max_power, Length: 8128, dtype: object

In [19]:
data[['max_power','max_power_unit']]=data['max_power'].str.split(expand=True)

In [20]:
data['max_power']

0           74
1       103.52
2           78
3           90
4         88.2
         ...  
8123     82.85
8124       110
8125      73.9
8126        70
8127        70
Name: max_power, Length: 8128, dtype: object

In [21]:
data['max_power']=data['max_power'].astype('float32')

ValueError: could not convert string to float: 'bhp'

In [22]:
data[data['max_power']=='bhp']

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,max_power_unit
4933,Maruti Omni CNG,2000,80000,100000,CNG,Individual,Manual,Second Owner,10.9 km/kg,796.0,bhp,NaN,8.0,None


In [23]:
import numpy as np

In [24]:
#숫자형 확인
def isFloat(value):
    try:
        num=float(value)
        return num
    except ValueError:
        return np.NaN


In [25]:
data['max_power']=data['max_power'].apply(isFloat)

In [28]:
data['max_power_unit'].value_counts()

bhp    7906
Name: max_power_unit, dtype: int64

In [26]:
data.drop('max_power_unit',axis=1, inplace=True)

In [27]:
data[['mileage','mileage_unit']]=data['mileage'].str.split(expand=True)

In [28]:
data['mileage']

0        23.4
1       21.14
2        17.7
3        23.0
4        16.1
        ...  
8123     18.5
8124     16.8
8125     19.3
8126    23.57
8127    23.57
Name: mileage, Length: 8128, dtype: object

In [29]:
data['mileage']=data['mileage'].astype('float32')

In [30]:
data['mileage_unit'].value_counts()

kmpl     7819
km/kg      88
Name: mileage_unit, dtype: int64

In [31]:
data['fuel'].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [33]:
def mile(x):
    if x['fuel']=='Petrol':
        return x['mileage'] / 80.43
    elif x['fuel']=='Diesel':
         return x['mileage'] / 73.56
    elif x['fuel']=='LPG':
         return x['mileage'] / 40.85
    else:
         return x['mileage'] / 44.23



In [34]:
data['mileage']=data.apply(mile,axis=1)

In [35]:
data.drop('mileage_unit',axis=1,inplace=True)

In [36]:
data['torque'].head()

0              190Nm@ 2000rpm
1         250Nm@ 1500-2500rpm
2       12.7@ 2,700(kgm@ rpm)
3    22.4 kgm at 1750-2750rpm
4       11.5@ 4,500(kgm@ rpm)
Name: torque, dtype: object

In [37]:
data['torque']=data['torque'].str.upper()

In [38]:
def torque_unit(x):
    if 'NM' in str(x):
        return 'Nm'
    elif 'KGM' in str(x):
        return 'kgm'


In [39]:
data['torque_unit']=data['torque'].apply(torque_unit)

In [40]:
data['torque_unit'].unique()

array(['Nm', 'kgm', None], dtype=object)

In [41]:
data[data['torque_unit'].isna()]['torque'].unique()

array([nan, '250@ 1250-5000RPM', '510@ 1600-2400', '110(11.2)@ 4800',
       '210 / 1900'], dtype=object)

In [42]:
data['torque_unit'].fillna('Nm',inplace=True)

In [43]:
string_example='12.7@ 2,700(KGM@ RPM)'
string_example[:4]

'12.7'

In [44]:
for i, j in enumerate(string_example):
    print(i,'번째 텍스트',j)

0 번째 텍스트 1
1 번째 텍스트 2
2 번째 텍스트 .
3 번째 텍스트 7
4 번째 텍스트 @
5 번째 텍스트  
6 번째 텍스트 2
7 번째 텍스트 ,
8 번째 텍스트 7
9 번째 텍스트 0
10 번째 텍스트 0
11 번째 텍스트 (
12 번째 텍스트 K
13 번째 텍스트 G
14 번째 텍스트 M
15 번째 텍스트 @
16 번째 텍스트  
17 번째 텍스트 R
18 번째 텍스트 P
19 번째 텍스트 M
20 번째 텍스트 )


In [45]:
for i, j in enumerate(string_example):
    if j not in '0123456789.':
        cut=i
        break


In [50]:
#처음 나오는 숫자 정보만 추출하여 return하는 함수
def split_num(x):
    x=str(x)
    for i, j in enumerate(x):
        if j not in '0123456789.':
            cut=i
            break
    return x[:cut]

In [51]:
data['torque']=data['torque'].apply(split_num)

UnboundLocalError: local variable 'cut' referenced before assignment

In [60]:
data['torque']

0         190
1         250
2        12.7
3        22.4
4        11.5
        ...  
8123    113.7
8124       24
8125      190
8126      140
8127      140
Name: torque, Length: 8128, dtype: object

In [52]:
data['torque']=data['torque'].astype('float64')

ValueError: could not convert string to float: ''

In [53]:
data['torque']=data['torque'].replace('',np.NaN)

In [54]:
data['torque']=data['torque'].astype('float64')

In [55]:
data['torque'].head()

0    190.0
1    250.0
2     12.7
3     22.4
4     11.5
Name: torque, dtype: float64

In [56]:
def torque_trans(x):
    if x['torque_unit']=='kgm':
        return x['torque'] * 9.8066
    else:
        return x['torque'] 

In [57]:
data['torque']=data.apply(torque_trans,axis=1)

In [58]:
data.drop('torque_unit',axis=1,inplace=True)

In [59]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


In [60]:
data['name']=data['name'].str.split(expand=True)[0]

In [61]:
data['name'].unique()

array(['Maruti', 'Skoda', 'Honda', 'Hyundai', 'Toyota', 'Ford', 'Renault',
       'Mahindra', 'Tata', 'Chevrolet', 'Fiat', 'Datsun', 'Jeep',
       'Mercedes-Benz', 'Mitsubishi', 'Audi', 'Volkswagen', 'BMW',
       'Nissan', 'Lexus', 'Jaguar', 'Land', 'MG', 'Volvo', 'Daewoo',
       'Kia', 'Force', 'Ambassador', 'Ashok', 'Isuzu', 'Opel', 'Peugeot'],
      dtype=object)

In [62]:
data['name']=data['name'].replace('Land','Land Rover')

In [63]:
data.isna().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026575
torque           0.027313
seats            0.027190
dtype: float64

In [64]:
data.dropna(inplace=True)
len(data)

7906

In [65]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti,2014,450000,145500,Diesel,Individual,Manual,First Owner,0.318108,1248.0,74.00,190.00000,5.0
1,Skoda,2014,370000,120000,Diesel,Individual,Manual,Second Owner,0.287384,1498.0,103.52,250.00000,5.0
2,Honda,2006,158000,140000,Petrol,Individual,Manual,Third Owner,0.220067,1497.0,78.00,124.54382,5.0
3,Hyundai,2010,225000,127000,Diesel,Individual,Manual,First Owner,0.312670,1396.0,90.00,219.66784,5.0
4,Maruti,2007,130000,120000,Petrol,Individual,Manual,First Owner,0.200174,1298.0,88.20,112.77590,5.0


In [66]:
data=pd.get_dummies(data,columns=['name','fuel','seller_type','transmission','owner'],drop_first=True)

In [67]:
data.head()

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0


In [68]:
#모델링 및 평가하기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(data.drop('selling_price',axis=1),data['selling_price'],test_size=0.2,random_state=100)

In [69]:
from sklearn.ensemble import RandomForestClassifier

In [70]:
model=RandomForestClassifier(random_state=100)
model.fit(X_train,y_train)
train_pred=model.predict(X_train)
test_pred=model.predict(X_test)

In [71]:
#RMSE 구해보기 
from sklearn.metrics import mean_squared_error
print("train_rmse:",mean_squared_error(y_train, train_pred),"test_rmse:",mean_squared_error(y_test, test_pred))

train_rmse: 343555628.08728653 test_rmse: 34589126927.38053


In [72]:
#K-Fold 교차검증
from sklearn.model_selection import KFold 

In [73]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
8124,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
8125,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
8126,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [74]:
data.reset_index(drop=True, inplace=True) #reindex

In [75]:
data

,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,name_Ashok,name_Audi,...,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,2014,450000,145500,0.318108,1248.0,74.00,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
1,2014,370000,120000,0.287384,1498.0,103.52,250.00000,5.0,0,0,...,1,0,0,1,0,1,0,1,0,0
2,2006,158000,140000,0.220067,1497.0,78.00,124.54382,5.0,0,0,...,0,0,1,1,0,1,0,0,0,1
3,2010,225000,127000,0.312670,1396.0,90.00,219.66784,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
4,2007,130000,120000,0.200174,1298.0,88.20,112.77590,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,320000,110000,0.230014,1197.0,82.85,113.70000,5.0,0,0,...,0,0,1,1,0,1,0,0,0,0
7902,2007,135000,119000,0.228385,1493.0,110.00,235.35840,5.0,0,0,...,1,0,0,1,0,1,1,0,0,0
7903,2009,382000,120000,0.262371,1248.0,73.90,190.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0
7904,2013,290000,25000,0.320419,1396.0,70.00,140.00000,5.0,0,0,...,1,0,0,1,0,1,0,0,0,0


In [76]:
kf=KFold(n_splits=5) #KFold 객체 생성

In [77]:
X=data.drop('selling_price',axis=1)

In [78]:
y=data['selling_price']

In [79]:
for i,j in kf.split(X):
    print(i,j)

[1582 1583 1584 ... 7903 7904 7905] [   0    1    2 ... 1579 1580 1581]
[   0    1    2 ... 7903 7904 7905] [1582 1583 1584 ... 3160 3161 3162]
[   0    1    2 ... 7903 7904 7905] [3163 3164 3165 ... 4741 4742 4743]
[   0    1    2 ... 7903 7904 7905] [4744 4745 4746 ... 6322 6323 6324]
[   0    1    2 ... 6322 6323 6324] [6325 6326 6327 ... 7903 7904 7905]


In [80]:
for train_index, test_index in kf.split(X):
    X_train,X_test=X.loc[train_index],X.loc[train_index]
    y_train,y_test=y[train_index],y[test_index]

In [81]:
train_rmse_total=[]
test_rmse_total=[]

In [89]:
for train_index, test_index in kf.split(X):
    X_train,X_test=X.loc[train_index],X.loc[test_index]
    y_train,y_test=y[train_index],y[test_index]
    model=RandomForestClassifier(random_state=100)
    model.fit(X_train,y_train)
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)

    train_rmse=mean_squared_error(y_train,train_pred) **0.5
    test_rmse=mean_squared_error(y_test,test_pred)**0.5
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [90]:
train_rmse_total

[19028.208885783548,
 19516.556077089976,
 20673.247557876555,
 19419.7977300783,
 20644.76329974385]

In [91]:
print ("train_rmse",sum(train_rmse_total)/5,"test_rmse",sum(test_rmse_total)/5) 

train_rmse 19856.514710114447 test_rmse 187324.89475198218


In [92]:
#파라메터 튜닝
train_rmse_total=[]
test_rmse_total=[]
for train_index, test_index in kf.split(X):
    X_train,X_test=X.loc[train_index],X.loc[test_index]
    y_train,y_test=y[train_index],y[test_index]
    model=RandomForestClassifier(n_estimators=300,max_depth=50,min_samples_split=5,min_samples_leaf=1,n_jobs=-1,random_state=100)
    #n_estimators 결정트리 개수
    #max_depth 트리의 최대 깊이
    #min_samples_split 해당 노드를 나눌것인지 말것인지를 노드 데이터 수를 기준으로 판단
    #min_samples_leaf 분리된 노드의 데이터에 최소 몇개의 데이터거 있어야 할지 기본값은 1
    #n_jobs 병렬처리에 사용하는 CPU 코어 수
    model.fit(X_train,y_train)
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)

    train_rmse=mean_squared_error(y_train,train_pred) **0.5
    test_rmse=mean_squared_error(y_test,test_pred)**0.5
    train_rmse_total.append(train_rmse)
    test_rmse_total.append(test_rmse)

In [93]:
print ("train_rmse",sum(train_rmse_total)/5,"test_rmse",sum(test_rmse_total)/5) 

train_rmse 39174.240063292484 test_rmse 197425.85982766422
